In [10]:
import openai


In [11]:
client = openai.Client()


In [13]:
vector_store = client.vector_stores.create(
    name="Tutor de Apostilas"
)


In [14]:
file = ["files/LLM.pdf"]
file_stream = [open(f, "rb") for f in file]
file_batch = client.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_stream
)


In [15]:
file_batch.status


'completed'

In [16]:
file_batch.file_counts


FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)

In [17]:
assistant = client.beta.assistants.create(
    name="Tutor Apostila",
    instructions="Você é um tutor especializado em tecnologias emergentes. \
        Você sabe responder perguntas sobre LLMs como OpenAI, HuggingFace, etc. \
            Caso você não encontre as resostas, seja sincero e fale que não sabe responder",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids":[vector_store.id]}},
    model="gpt-4-turbo-preview"
)


In [18]:
pergunta = "Conforme o documento, o que é o OpenAI?"


In [19]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)


In [20]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)


In [21]:
# Aguarda a thread rodar
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )


In [22]:
# Verifica a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")


SyncCursorPage[Message](data=[Message(id='msg_Fy010CPAvhmw5Fc9GCtKZlzV', assistant_id='asst_fjXnFNLsn7QQblyvyTY3FnZr', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=239, file_citation=FileCitation(file_id='file-BPWSqEx8d5YgdrHKNWtTiZ'), start_index=226, text='【4:0†LLM.pdf】', type='file_citation')], value='A OpenAI é descrita no documento como uma organização que, em 2020, lançou o GPT-3, se tornando o maior modelo com 175 bilhões de parâmetros e estabelecendo um novo benchmark de desempenho para tarefas relacionadas à linguagem【4:0†LLM.pdf】.'), type='text')], created_at=1745284613, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_td8sIpN1KIkzVFe90u2mhSTc', status=None, thread_id='thread_hUxqSPFfOar0u6sFO8GI5YU9'), Message(id='msg_Pxl9nHwuvqM6s0sFGjkCJGrQ', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotati

In [23]:
print(mensagens.data[0].content[0].text.value)


A OpenAI é descrita no documento como uma organização que, em 2020, lançou o GPT-3, se tornando o maior modelo com 175 bilhões de parâmetros e estabelecendo um novo benchmark de desempenho para tarefas relacionadas à linguagem【4:0†LLM.pdf】.
